In [11]:
from requests import get
from json import loads
from urllib.parse import unquote
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
topic_url_format = 'https://api.duckduckgo.com/?q={search_term}&format=json&pretty=1'

In [3]:
topic_seed = ['user experience design']

In [4]:
responses = []

In [5]:
for topic in topic_seed:
    responses.append(get(topic_url_format.format(search_term=topic)))

In [6]:
content = loads(responses[0].content)

In [7]:
topic_rows = []
related_topic_rows = []
for idx, response in enumerate(responses):
    content = loads(response.content)
    data = {}
    data['topic'] = content['Heading'].lower()
    data['abstract'] = content['Abstract']
    data['document_type'] = content['Type']
    data['topic_wiki_url'] = content['AbstractURL']
    topic_rows.append(data)   
    
    for result in content['RelatedTopics']:
        row = {}
        row['related_topic'] = unquote(result['FirstURL'].replace(
            'https://duckduckgo.com/', '').replace('c/', '').replace('_', ' ')).lower()
        row['related_topic_text'] = result['Text']
        row.update(data)
        related_topic_rows.append(row)
topics = pd.DataFrame(topic_rows)
related_topics = pd.DataFrame(related_topic_rows)

In [8]:
topics

,abstract,document_type,topic,topic_wiki_url
0,User experience design is the process of manip...,A,user experience design,https://en.wikipedia.org/wiki/User_experience_...


In [9]:
related_topics

,abstract,document_type,related_topic,related_topic_text,topic,topic_wiki_url
0,User experience design is the process of manip...,A,action research,Action research - Action research is a philoso...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
1,User experience design is the process of manip...,A,activity-centered design,Activity-centered design - Activity-centered d...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
2,User experience design is the process of manip...,A,customer experience,"Customer experience - In commerce, customer ex...",user experience design,https://en.wikipedia.org/wiki/User_experience_...
3,User experience design is the process of manip...,A,design thinking,Design thinking - Design thinking refers to th...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
4,User experience design is the process of manip...,A,paper prototyping,Paper prototyping - In human–computer interact...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
5,User experience design is the process of manip...,A,participatory design,Participatory design - Participatory design is...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
6,User experience design is the process of manip...,A,web design,Web design,user experience design,https://en.wikipedia.org/wiki/User_experience_...
7,User experience design is the process of manip...,A,software development process,Software development process,user experience design,https://en.wikipedia.org/wiki/User_experience_...
8,User experience design is the process of manip...,A,user interfaces,User interfaces,user experience design,https://en.wikipedia.org/wiki/User_experience_...
9,User experience design is the process of manip...,A,human–computer interaction,Human–computer interaction,user experience design,https://en.wikipedia.org/wiki/User_experience_...


In [33]:
from time import sleep
topic_df = pd.DataFrame()
related_topic_df = pd.DataFrame()
for depth in range(2):
    if not depth:
        topics = topic_seed
    related_topics = []
    topic_rows = []
    related_topic_rows = []
    for topic in topics:
        try:
            print(topic)
            sleep(0.1)
            response = get(topic_url_format.format(search_term=topic))
            content = loads(response.content)
            data = {}
            data['topic'] = content['Heading'].lower()
            data['abstract'] = content['Abstract']
            data['document_type'] = content['Type']
            data['topic_wiki_url'] = content['AbstractURL']
            topic_rows.append(data)   

            for result in content['RelatedTopics']:
                row = {}
                row['depth'] = depth
                row['related_topic'] = unquote(result['FirstURL'].replace(
                    'https://duckduckgo.com/', '').replace('c/', '').replace('_', ' ')).lower()
                row['related_topic_text'] = result['Text']
                row.update(data)
                related_topic_rows.append(row)
                related_topics.append(row['related_topic'])
        except KeyError as e:
            print('Error on topic: ' + topic + ' : ' + str(e))
    topics = related_topics
    topic_df = topic_df.append(pd.DataFrame(topic_rows))
    related_topic_df = related_topic_df.append(pd.DataFrame(related_topic_rows))

user experience design
action research
activity-centered design
customer experience
design thinking
paper prototyping
participatory design
web design
software development process
user interfaces
human–computer interaction
design
Error on topic: design : 'FirstURL'


In [30]:
related_topic_df

,abstract,document_type,related_topic,related_topic_text,topic,topic_wiki_url
0,User experience design is the process of manip...,A,action research,Action research - Action research is a philoso...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
1,User experience design is the process of manip...,A,activity-centered design,Activity-centered design - Activity-centered d...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
2,User experience design is the process of manip...,A,customer experience,"Customer experience - In commerce, customer ex...",user experience design,https://en.wikipedia.org/wiki/User_experience_...
3,User experience design is the process of manip...,A,design thinking,Design thinking - Design thinking refers to th...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
4,User experience design is the process of manip...,A,paper prototyping,Paper prototyping - In human–computer interact...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
5,User experience design is the process of manip...,A,participatory design,Participatory design - Participatory design is...,user experience design,https://en.wikipedia.org/wiki/User_experience_...
6,User experience design is the process of manip...,A,web design,Web design,user experience design,https://en.wikipedia.org/wiki/User_experience_...
7,User experience design is the process of manip...,A,software development process,Software development process,user experience design,https://en.wikipedia.org/wiki/User_experience_...
8,User experience design is the process of manip...,A,user interfaces,User interfaces,user experience design,https://en.wikipedia.org/wiki/User_experience_...
9,User experience design is the process of manip...,A,human–computer interaction,Human–computer interaction,user experience design,https://en.wikipedia.org/wiki/User_experience_...


In [34]:
related_topic_df.to_csv('topic_database.csv')

In [17]:
a = [1]
for i in a:
    a.append(i)
    if len(a) > 10:
        break
a

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [31]:
related_topics

['social science methodology',
 'design',
 'customer knowledge',
 'customer success',
 'customer data platform',
 'the experience economy',
 'user experience',
 'customer service',
 'branding terminology',
 'services marketing',
 'telecommunications systems',
 'product management',
 'brand management',
 'creativity techniques',
 'design-based learning',
 'design methods',
 'empathic design',
 'lateral thinking',
 'problem structuring methods',
 'problem solving',
 'design',
 'prototype',
 'software prototyping',
 'user-centered design',
 'cardboard modeling',
 'software design',
 'user interfaces',
 'computer-supported cooperative work',
 'design thinking',
 'participatory action research',
 'permaculture',
 'public participation',
 'service design',
 'product development',
 'innovation',
 'design',
 'web design',
 'communication design',
 'software development process',
 'systems development life cycle',
 'computer-aided software engineering',
 'outline of software engineering',
 'ope